In [1]:
require 'nn'
require 'rnn'
require 'optim'
require 'hdf5'

In [2]:
opt = {
    useGPU = 1,
    loadModel = 0,
    loadModelPath = "models/quoraNetComp_batchSize_512_Lr_0.001_learningRateDecay_0_optimizer_adam_weightDecay_0.0005_stepDecayStep_5_epoch_15.t7",
    
    batchSize = 512, --IT should be less.. But to make it faster good range 8-128 (2^i)
    seqLen = 55,  --Assumed maximum words in sentance
    wordVecSize = 50,
    lstmOutputSize = 400,  --dont change

    useDropout = 1,
    dropoutProbability = 0.5,
    useBN = 1,                    --Will take care of wieght intilization problem
    useLeakyRelu = 0,
    leakyReluSlope = 0.2,

    useLoss = "SoftMax", -- "SoftMax" or "SVM" -- TAKE CARE LABEL SHOULD BE CHANGED TO 1 and -1 for "SVM"                      
    svmMargin = 1,
    optimizer = "adam",  --"sgd" or "adam"
    
    trainDataPath = "/dataset/train10000to60000.h5py",
    evalDataPath = "/dataset/train0to10000.h5py",  
    trainDataSize = 44020,
    valDataSize = 8793,  --Check
    maxTrainDataSize  = 50000,    -- Maximum to run while training
    maxValDataSize = 50000,  
    
    learningRate = 0.001,
    learningRateDecay = 0,--1e-5,  -- 1/t decay.. Set to 0 if setting step decay to true
    weightDecay = 0.01,     -- l2 regularizer
    momentum = 0.9,
    nesterov = true,
    beta1 = 0.9,
    beta2 = 0.999,
    epsilon = 1e-8,
    stepDecay = false,
    stepDecayRate = 0.5,
    stepDecayStep = 1,
    
    nEpochs = 25,
    validateAtEach  = 1,
    saveAtEach = 10,
    savePath ="models/quoraNetCompFinal",
    trainingStatsSavePath = "trainingStats/quoraNetCompFinal",
    
    
    wieghtIntalization = "" -- "yaanLeCun" , "Xavier" or "KaimingHe"  (NOT IMPLEMENTED FOR NOW)
            
}


trainingStats = {valLoss={}, trainLoss={}, valAccuracy={}, trainAccuracy={}, learningRate = {}, options = opt}

In [3]:
if opt.useGPU == 0 then
    torch.setdefaulttensortype('torch.FloatTensor')
    print ("CPU Mode")
else
    require 'cutorch'
    require 'cunn'
    print ("GPU Mode")
    --torch.setdefaulttensortype('torch.CudaTensor')
end

GPU Mode	


# LSTM Many-to-One Model

In [4]:
fullyConnectedBlock = function(nInput, nOutput, blockUseDrop, blockDropP, blockUseBN, blockUseLeakyRelu, leakySlope)
    block = nn.Sequential()
    block:add(nn.Linear(nInput, nOutput))
    
    if blockUseBN == 1 then
        block:add(nn.BatchNormalization(nOutput))
    end
    
    if blockUseLeakyRelu == 1 then
        block:add(nn.LeakyReLU(leakySlope, True))
    else
        block:add(nn.ReLU(True))
    end
    
    if blockUseDrop == 1 then
        block:add(nn.Dropout(blockDropP))
    end
    
    return block
end

In [5]:
if opt.loadModel == 0 then
    print("creating quoraNet...")
    LSTM = nn.FastLSTM(opt.wordVecSize, opt.lstmOutputSize)
    rnnSen1 = nn.Sequential()
       :add(nn.SplitTable(1,2))
       :add(nn.Sequencer(LSTM))
       :add(nn.SelectTable(-1)) -- LSTM
                                -- Many To One Mapping

    --rnnSen2 = rnnSen1:clone('weight','bias');  -- Share same wieght and bias
    rnnSen2 = rnnSen1:clone();   --Now do not share wieghts

    parallelLSTMs = nn.ParallelTable()
    parallelLSTMs:add(rnnSen1)
    parallelLSTMs:add(rnnSen2)

    quoraNet = nn.Sequential()
    quoraNet:add(parallelLSTMs)
    quoraNet:add(nn.JoinTable(2))

    quoraNet:add(fullyConnectedBlock(800,256,opt.useDropout,opt.dropoutProbability,opt.useBN,opt.useLeakyRelu,opt.leakyReluSlope))
    quoraNet:add(fullyConnectedBlock(256,256,opt.useDropout,opt.dropoutProbability,opt.useBN,opt.useLeakyRelu,opt.leakyReluSlope))
    quoraNet:add(fullyConnectedBlock(256,256,opt.useDropout,opt.dropoutProbability,opt.useBN,opt.useLeakyRelu,opt.leakyReluSlope))
    quoraNet:add(fullyConnectedBlock(256,256,opt.useDropout,opt.dropoutProbability,opt.useBN,opt.useLeakyRelu,opt.leakyReluSlope))
    quoraNet:add(nn.Linear(256,1))
else
    print("loading quoraNet from "..opt.loadModelPath)
    quoraNet = torch.load(opt.loadModelPath)
end

criterion = nil
if opt.useLoss == "SoftMax" then
    print("Using SoftMax")
    
    if opt.loadModel == 0 then
        print("adding sigmoid")
        quoraNet:add(nn.Sigmoid())   --Dont add again if already loaded
    end
    
    criterion = nn.BCECriterion()
elseif opt.useLoss == "SVM" then      -- TAKE CARE LABEL SHOULD BE CHANGED TO 1 and -1
    print("Using SVM")
    crtierion = nn.MarginCriterion(opt.svmMargin)
else
    error("Provide Correct Loss")
end

if opt.useGPU == 1 then
    print("converting model to cuda...")
    quoraNet:cuda()
    criterion:cuda()
end

params,gradParams = quoraNet:getParameters()

creating quoraNet...	


Using SoftMax	
adding sigmoid	
converting model to cuda...	


# Data Loader

In [6]:
dataTrain =   hdf5.open(opt.trainDataPath,'r')
dataEval =  hdf5.open(opt.evalDataPath,'r')

nTrainBatches = math.floor(math.min(opt.trainDataSize, opt.maxTrainDataSize) / opt.batchSize)
nValBatches = math.floor(math.min(opt.valDataSize, opt.maxValDataSize) / opt.batchSize)

print(("training batches: %d"):format(nTrainBatches))
print(("validation batches: %d"):format(nValBatches))

training batches: 85	
validation batches: 17	


# Confusiton Matrix

In [7]:
classes = {'0', '1'}
trainConfusion = optim.ConfusionMatrix(classes) --save Confustion Matrix
valConfusion = optim.ConfusionMatrix(classes) --save Confustion Matrix
trainValidTable = {}
valValidTable = {}


function updateConfustion(confusion, class0or1, outputProb, validTable)
    output = torch.ones(2)
    if outputProb[1] < 0.5 then
        output[2] = 0 
    else 
        output[1] = 0
    end
    
    target = torch.ones(2)
    if class0or1[1] == 0 then
        target[2] = 0
    else
        target[1] = 0
    end
    
    if output[1] == target[1] then
        table.insert(validTable,1)
    else
        table.insert(validTable,0)
    end
    
    confusion:add(output, target)
end

function getAccuracy(validTable)
    len = #validTable 
    if len == 0 then
        return 0
    else
        sum = 0
        for i = 1,len do
            sum = sum + validTable[i]
        end
        return sum/len
    end
end   

# TRAIN

In [8]:
sgdParam = nil
adamParam = nil

function updateSGDParam(opt)	
    --print("updating sgd param")
    sgdParam = {
        learningRate = opt.learningRate,
        learningRateDecay = opt.learningRateDecay,  -- 1/t decay
        weightDecay = opt.weightDecay,
        momentum = opt.momentum,
        nesterov = opt.nesterov,
        dampening = 0  --it should be zero for nestrov
    }
end

function updateAdamParam(opt)
    --print("updating adam param")
    adamParam = {
        learningRate = opt.learningRate,
        learningRateDecay = opt.learningRateDecay,  --1/t decay
        weightDecay = opt.weightDecay,
        beta1 = opt.beta1,
        beta2 = opt.beta2,
        epsilon = opt.epsilon
    }
end

function stepDecayLearningRate(opt)
    opt.learningRate = opt.stepDecayRate*opt.learningRate
     print("new learning rate "..opt.learningRate)
    if opt.optimizer == 'sgd' then
        updateSGDParam(opt)
    elseif opt.optimizer == 'adam' then
        updateAdamParam(opt)
    else
        error('Provide a correct optimizer')
    end
end

In [9]:
step = function()
    
    quoraNet:training()
    nn.FastLSTM.bn = true
    
    local currentLoss = 0
    -- shuffle at each epoch
    local shuffle = torch.randperm(opt.trainDataSize)

    for t = 1, nTrainBatches do 
                             -- For no of batches(seq len)
        local feval = function(params_new)
            
            if params ~= params_new then params:copy(params_new) end   
            gradParams:zero()

            --quoraNet:forget() --done uncomment, sequencer will itself call it before each call to forward
            collectgarbage()
            
            local question1 = torch.ones(opt.batchSize, opt.seqLen, opt.wordVecSize) --Provide size here
            local question2 = torch.ones(opt.batchSize, opt.seqLen, opt.wordVecSize)
            local label = torch.ones(opt.batchSize)
            
            local idx = 1
            local st = opt.batchSize*(t-1)+1
            local fn = opt.batchSize*t
            
            for i = st, fn do
                question1[idx] = dataTrain:read('/question1'):partial({shuffle[i],shuffle[i]},{1,opt.seqLen},{1,opt.wordVecSize})
                question2[idx] = dataTrain:read('/question2'):partial({shuffle[i],shuffle[i]},{1,opt.seqLen},{1,opt.wordVecSize})
                label[idx] = dataTrain:read('/label'):partial({shuffle[i],shuffle[i]})
                idx = idx + 1
            end
            
            label = label:resize(opt.batchSize,1)
            -- Add Normaliztion here-- question1:add(-mean[i]) -- mean subtraction
            
            if opt.useGPU == 1 then
                question1 = question1:cuda()
                question2 = question2:cuda()
                label = label:cuda()
            end

            --FORWARD PASS
            local outputs = quoraNet:forward({question1,question2})
            local loss = criterion:forward(outputs, label)
            
            --BACKWARD PASS
            local dloss_doutput = criterion:backward(outputs, label)
            quoraNet:backward({question1,question2}, dloss_doutput)
            
            for i  = 1,opt.batchSize do
                updateConfustion(trainConfusion, label[i], outputs[i], trainValidTable)
            end

            return loss,gradParams
        end
            
        if opt.optimizer == 'sgd' then
            updateSGDParam(opt)
            _ , fs = optim.sgd(feval,params,sgdParam)

        elseif opt.optimizer == 'adam' then
            updateAdamParam(opt)
            _ , fs = optim.adam(feval,params,adamParam)

        else
            error('unknown optimizer')
        end

        currentLoss = currentLoss + fs[1]
    end

    return currentLoss/nTrainBatches
end


# Validation

In [10]:
eval = function()
    
    local currentLoss = 0
    
    quoraNet:evaluate()
    nn.FastLSTM.bn = false
    
    for t = 1,nValBatches do    --Evaluation Batch Size
        
        collectgarbage()
        local st = opt.batchSize*(t-1)+1
        local fn = opt.batchSize*t
        
        local question1 = dataEval:read('/question1'):partial({st,fn},{1,opt.seqLen},{1,opt.wordVecSize})
        local question2 = dataEval:read('/question2'):partial({st,fn},{1,opt.seqLen},{1,opt.wordVecSize})
        local label = dataEval:read('/label'):partial({st,fn})
        label = label:float():resize(opt.batchSize,1)
        
        -- Add Normaliztion here-- question1:add(-mean[i]) -- mean subtraction
        if opt.useGPU == 1 then
                question1 = question1:cuda()
                question2 = question2:cuda()
                label = label:cuda()
        end

        
       --FORWARD PASS
        local outputs = quoraNet:forward({question1,question2})
        local loss = criterion:forward(outputs, label)
        
        for i  = 1,opt.batchSize do
            updateConfustion(valConfusion, label[i], outputs[i], valValidTable)
        end
       
        currentLoss = currentLoss + loss
    end

    return currentLoss/nValBatches
end

# Controller

In [11]:
--- timer
local epoch_tm = torch.Timer()
local tm = torch.Timer()

for epoch = 1,opt.nEpochs do         -- FOR EPOCHS
    
    --StepDecayLearningRate
    if opt.stepDecay == true then
        if epoch%opt.stepDecayStep == 0 then
            print("step decay learning rate at ephoch: "..epoch)
            stepDecayLearningRate(opt)
        end
    end
    
    epoch_tm:reset()
    trainConfusion:zero()
    trainValidTable = {}
    
    local loss = step()
    local trainAcc = getAccuracy(trainValidTable)
    print(('End of epoch %d / %d \t Training loss: %4f \t Training accuracy: %0.3f \t Time Taken: %.3f'):
        format(epoch, opt.nEpochs, loss, trainAcc, epoch_tm:time().real))
    table.insert(trainingStats.trainLoss, loss)
    table.insert(trainingStats.trainAccuracy,  trainAcc)
    
    --print(trainConfusion)
   
    
    if epoch%opt.validateAtEach == 0 then   
      
        epoch_tm:reset()
        valConfusion:zero()
        valValidTable = {}
        
        local validation_loss = eval()
        local valAcc = getAccuracy(valValidTable)
        
        print(('Validation after epoch %d  \t Validation loss: %4f \t Validation accuracy: %0.3f  \t Time Taken: %.3f'):
        format(epoch, validation_loss, valAcc, epoch_tm:time().real))
        table.insert(trainingStats.valLoss, validation_loss)  
        table.insert(trainingStats.valAccuracy, valAcc)
     
        --print(valConfusion)
    end

    if epoch%opt.saveAtEach == 0  then
      print('saving network at stage'..epoch..'...')
      torch.save(opt.savePath..'_batchSize_'..opt.batchSize..'_Lr_'..opt.learningRate..'_learningRateDecay_'..opt.learningRateDecay..'_optimizer_'..opt.optimizer..'_weightDecay_'..opt.weightDecay..'_stepDecayStep_'..opt.stepDecayStep..'_epoch_'..epoch..'.t7', quoraNet:clearState())
      torch.save(opt.trainingStatsSavePath..'_batchSize_'..opt.batchSize..'_Lr_'..opt.learningRate..'_learningRateDecay_'..opt.learningRateDecay..'_optimizer_'..opt.optimizer..'_weightDecay_'..opt.weightDecay..'_stepDecayStep_'..opt.stepDecayStep..'_epoch_'..epoch..'.t7', trainingStats)
    end

end

torch.save(opt.trainingStatsSavePath..'_batchSize_'..opt.batchSize..'_Lr_'..opt.learningRate..'_learningRateDecay_'..opt.learningRateDecay..'_optimizer_'..opt.optimizer..'_weightDecay_'..opt.weightDecay..'_stepDecayStep_'..opt.stepDecayStep..'_final.t7', trainingStats)
print(('Total Time Taken: %.3f'):
        format(tm:time().real))
print("\n\n\n")
print("trainConfusion: ")
print(trainConfusion)
print("\n\n\n")
print("valConfusion: ")
print(valConfusion)
print("\n\n\n")
print("trainingStats.trainLoss: ")
print(trainingStats.trainLoss)
print("\n\n\n")
print("trainingStats.trainAccuracy: ")
print(trainingStats.trainAccuracy)
print("\n\n\n")
print("trainingStats.valLoss: ")
print(trainingStats.valLoss)
print("\n\n\n")
print("trainingStats.valAccuracy: ")
print(trainingStats.valAccuracy)

dataTrain:close()
dataEval:close()

End of epoch 1 / 25 	 Training loss: 0.655250 	 Training accuracy: 0.616 	 Time Taken: 82.064	


Validation after epoch 1  	 Validation loss: 0.634926 	 Validation accuracy: 0.642  	 Time Taken: 2.371	


End of epoch 2 / 25 	 Training loss: 0.623192 	 Training accuracy: 0.657 	 Time Taken: 47.433	


Validation after epoch 2  	 Validation loss: 0.653546 	 Validation accuracy: 0.625  	 Time Taken: 2.312	


End of epoch 3 / 25 	 Training loss: 0.615361 	 Training accuracy: 0.665 	 Time Taken: 47.509	


...

Validation after epoch 24  	 Validation loss: 0.591917 	 Validation accuracy: 0.685  	 Time Taken: 2.320	


End of epoch 25 / 25 	 Training loss: 0.580067 	 Training accuracy: 0.698 	 Time Taken: 47.374	


Validation after epoch 25  	 Validation loss: 0.608005 	 Validation accuracy: 0.669  	 Time Taken: 2.317	
Total Time Taken: 1278.073	



	
trainConfusion: 	
ConfusionMatrix:
[[   20824    5514]   79.064% 	[class: 0]
 [    7644    9538]]  55.512% 	[class: 1]
 + average row correct: 67.288026213646% 
 + average rowUcol correct (VOC measure): 51.652263104916% 
 + global correct: 69.765625%
{
  _mat_flat : LongTensor - size: 4
  valids : FloatTensor - size: 2
  mat : LongTensor - size: 2x2
  averageUnionValid : 0.51652263104916


  _targ_idx : LongTensor - size: 1
  averageValid : 0.67288026213646
  classes : 
    {
      1 : 0
      2 : 1
    }
  _prediction : FloatTensor - size: 2
  _pred_idx : LongTensor - size: 1
  nclasses : 2
  _max : FloatTensor - size: 1
  _target : FloatTensor - size: 2
  unionvalids : FloatTensor - size: 2
  totalValid : 0.69765625
}



	
valConfusion: 	
ConfusionMatrix:
[[    4511     756]   85.646% 	[class: 0]
 [    2122    1315]]  38.260% 	[class: 1]
 + average row correct: 61.953295767307% 
 + average rowUcol correct (VOC measure): 46.206003427505% 
 + global correct: 66.934742647059%
{
  _mat_flat : LongTensor - size: 4
  valids : FloatTensor - size: 2
  mat : LongTensor - size: 2x2
  averageUnionValid : 0.46206003427505
  _targ_idx : LongTensor - size: 1
  averageValid : 0.61953295767307
  classes : 
    {
      1 : 0
      2 : 1
    }
  _prediction : FloatTensor - size: 2
  _pred_idx : LongTensor - size: 1
  nclasses : 2
  _max : FloatTensor - size: 1
  _target : FloatTensor - 

  17 : 0.61637305161532
  18 : 0.59472835063934
  19 : 0.60456253500546
  20 : 0.5976347747971
  21 : 0.59437658155666
  22 : 0.6095895521781
  23 : 0.61185369421454
  24 : 0.59191671539755
  25 : 0.60800486101824
}



	
trainingStats.valAccuracy: 	
{
  1 : 0.64234834558824
  2 : 0.62522977941176
  3 : 0.61937040441176
  4 : 0.626953125
  5 : 0.62431066176471
  6 : 0.65682444852941
  7 : 0.66096047794118
  8 : 0.65625
  9 : 0.66567095588235
  10 : 0.66107536764706
  11 : 0.65866268382353
  12 : 0.65590533088235
  13 : 0.67279411764706
  14 : 0.66061580882353
  15 : 0.65303308823529
  16 : 0.66727941176471
  17 : 0.65521599264706
  18 : 0.68347886029412
  19 : 0.67221966911765
  20 : 0.67899816176471
  21 : 0.68072150735294
  22 : 0.66394761029412
  23 : 0.66486672794118
  24 : 0.68520220588235
  25 : 0.66934742647059
}


# Ploting

In [12]:
Plot = require ("itorch.Plot")
 
x = {}
for i = 1,opt.nEpochs do
    table.insert(x,i)
end
 
 
local plot = Plot()
plot:line(x, trainingStats.trainLoss ,'red' ,'trainLoss')
plot:line(x, trainingStats.valLoss,'blue','valLoss')
plot:xaxis('EPOCHS')
plot:legend(true):title('Overfitting Check: LOSS')
plot:draw() -- or plot:draw() or plot:html()

In [13]:
local plot = Plot()
plot:line(x, trainingStats.trainAccuracy ,'red' ,'trainAccuracy')
plot:line(x, trainingStats.valAccuracy,'blue','valAccuracy')
plot:xaxis('EPOCHS')
plot:legend(true):title('Overfitting Check: Accuracy')
plot:draw() -- or plot:draw() or plot:html()